# pyMMF: Benchmark test for straights step index fibers

In [1]:
import pyMMF
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
rc('figure', figsize=(18,9))
rc('text', usetex=True)
from IPython.display import display, Math

## Fiber parameters

In [2]:
## Parameters
NA = 0.15
radius = 10. # in microns
areaSize = 3.5*radius # calculate the field on an area larger than the diameter of the fiber
npoints = 2**8 # resolution of the window
n1 = 1.45
wl = 0.6328 # wavelength in microns

## Instantiate the solver

In [3]:
# Create the fiber object
profile = pyMMF.IndexProfile(npoints = npoints, areaSize = areaSize)

# Initialize the index profile
profile.initStepIndex(n1=n1,a=radius,NA=NA)

# Instantiate the solver
solver = pyMMF.propagationModeSolver()

# Set the profile to the solver
solver.setIndexProfile(profile)

# Set the wavelength
solver.setWL(wl)

# Estimate the number of modes for a graded index fiber
Nmodes_estim = pyMMF.estimateNumModesSI(wl,radius,NA,pola=1)

print(f"Estimated number of modes using the V number = {Nmodes_estim}")

2024-07-09 00:30:36,041 - pyMMF.core [DEBUG  ]  Debug mode ON.


Estimated number of modes using the V number = 56


## Semi-analytical solution

In [4]:
modes_semianalytical = solver.solve(mode = 'SI', curvature = None)

2024-07-09 00:30:36,048 - pyMMF.solv [INFO   ]  Finding the propagation constant of step index fiber by numerically solving the dispersion relation.
/opt/miniconda/envs/pymmf/lib/python3.12/site-packages/pyMMF/solvers/SI.py:92: RuntimeWarning: invalid value encountered in sqrt
  w = np.sqrt(v**2 - u**2)
2024-07-09 00:30:36,248 - pyMMF.solv [INFO   ]  Found 59 modes in 0.20 seconds.
2024-07-09 00:30:36,248 - pyMMF.solv [INFO   ]  Finding analytical LP mode profiles associated to the propagation constants.
2024-07-09 00:30:40,609 - pyMMF.solv [INFO   ]  Found 59 LP mode profiles in 0.1 minutes.
2024-07-09 00:30:40,611 - pyMMF.core [DEBUG  ]  Mode data stored in memory.


## Numerical calculations

### 2d Eigenvalue solver: 
See the [tutorial](https://www.wavefrontshaping.net/post/id/3) 
for more information.

In [5]:
modes_eig = solver.solve(nmodesMax=Nmodes_estim+10,
                         boundary = 'close', 
                         mode = 'eig', 
                         curvature = None, 
                         propag_only=True)

2024-07-09 00:30:40,618 - pyMMF.solv [INFO   ]  Solving the spatial eigenvalue problem for mode finding.
2024-07-09 00:30:40,620 - pyMMF.solv [INFO   ]  Use close boundary condition.
2024-07-09 00:33:35,657 - pyMMF.solv [INFO   ]  Solver found 59 modes is 175.04 seconds.
2024-07-09 00:33:35,668 - pyMMF.core [DEBUG  ]  Mode data stored in memory.


### Radial solver: 
See the [tutorial](https://www.wavefrontshaping.net/post/id/66)
and the article [Learning and avoiding disorder in multimode fibers](https://arxiv.org/abs/2010.14813) 
for more information.

In [6]:
modes_radial = solver.solve(mode = 'radial',  
                         propag_only=True,
                         N_beta_coarse = int(1e3))

2024-07-09 00:33:35,689 - pyMMF.solv [INFO   ]  Found 5 radial mode(s) for m=0
2024-07-09 00:33:35,690 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 1
2024-07-09 00:33:35,697 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 2
2024-07-09 00:33:35,704 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 3
2024-07-09 00:33:35,709 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 4
2024-07-09 00:33:35,716 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 5
2024-07-09 00:33:35,729 - pyMMF.solv [INFO   ]  Found 4 radial mode(s) for m=1
2024-07-09 00:33:35,729 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 1
2024-07-09 00:33:35,737 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 2
2024-07-09 00:33:35,744 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 3
2024-07-09 00:33:35,751 - pyMMF.solv [INFO   ]  Searching propagation constant for |l| = 4
2024-07-09 00:33:35,766

## Comparing results

In [7]:
# Sort the modes
modes = {}
idx = np.flip(np.argsort(modes_semianalytical.betas), axis=0)
modes['SA'] = {'betas':np.array(modes_semianalytical.betas)[idx],'profiles':[modes_semianalytical.profiles[i] for i in idx]}
idx = np.flip(np.argsort(modes_eig.betas), axis=0)
modes['eig'] = {'betas':np.array(modes_eig.betas)[idx],'profiles':[modes_eig.profiles[i] for i in idx]}
idx = np.flip(np.argsort(modes_eig.betas), axis=0)
modes['radial'] = {'betas':np.array(modes_radial.betas)[idx],'profiles':[modes_radial.profiles[i] for i in idx]}


def sort(a):
    return np.flip(np.sort(a),axis=0)




plt.figure(); 
plt.plot(sort(np.real(modes_eig.betas)),
         label='Numerical simulations (eigenvalue solver)',
         linewidth=2.)
plt.plot(sort(np.real(modes_semianalytical.betas)),
         'r--',
         label='Semi-analytical',
         linewidth=2.)
plt.plot(sort(np.real(modes_radial.betas)),
         'go',
         label='Numerical simulations (radial solver)',
         linewidth=2.)

plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
plt.title(r'Semi-analytical VS numerical simulations' ,fontsize = 30)
plt.ylabel(r'Propagation constant $\beta$ (in $\mu$m$^{-1}$)', fontsize = 25)
plt.xlabel(r'Mode index', fontsize = 25)
plt.legend(fontsize = 22,loc='upper right')
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1800x900 with 1 Axes>

## Comparing numerical solutions to LP modes

In [8]:
imode = 15
plt.figure()
plt.subplot(131)
plt.imshow(np.abs(modes['SA']['profiles'][imode].reshape([npoints]*2)))
plt.gca().set_title("Ideal LP mode",fontsize=25)
plt.axis('off')
plt.subplot(132)
plt.imshow(np.abs(modes['eig']['profiles'][imode].reshape([npoints]*2)))
plt.gca().set_title("Numerical simulations",fontsize=25)
plt.axis('off')
plt.subplot(133)
plt.imshow(np.abs(modes['radial']['profiles'][imode].reshape([npoints]*2)))
plt.gca().set_title("Numerical simulations",fontsize=25)
plt.axis('off')

(np.float64(-0.5), np.float64(255.5), np.float64(255.5), np.float64(-0.5))

Error in callback <function _draw_all_if_interactive at 0x7b4eb0379da0> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1800x900 with 3 Axes>